In [1]:
import csv
import os
import json
import pandas as pd
import plotly.express as px
import numpy as np

In [2]:
from app.flipside.models import Flipside
from app.constants import oracle_feeds_lookup, oracle_feeds, phase_windows, filenames_filtered, timestamp_windows

In [3]:
filenames = {    
    "oracles": "chainlink_all_oracle_results3",
}



In [ ]:
fs = Flipside(filenames)

In [5]:
window_start = 14053849 - 10
window_end = 16074000 + 10

In [ ]:
oracles = fs.apply_window('oracles', window_start, window_end)
print(oracles.keys())
oracles.FEED_ADDRESS.unique()

In [7]:
crv_address = oracle_feeds['CRV'].lower()
dai_address = oracle_feeds['DAI'].lower()

oracle_crv = oracles[oracles["FEED_ADDRESS"].isin([crv_address])]


In [ ]:
oracle_crv['ADJUSTED_PRICE'] = oracle_crv['LATEST_ANSWER_UNADJ'] / 10**18
oracle_crv['REVERSE_PRICE'] = 1/ oracle_crv['ADJUSTED_PRICE']

In [ ]:
oracle_crv['REVERSE_PRICE'].tail()

In [ ]:
fig = px.line(oracle_crv, 
                 x=oracle_crv.BLOCK_TIMESTAMP,
                 y=oracle_crv.ADJUSTED_PRICE,
                 color="FEED_NAME",
                title="CRV/ETH Price")
# fig.update_layout(autotypenumbers='convert types')
fig.show()

In [ ]:
fig = px.line(oracle_crv, 
                 x=oracle_crv.BLOCK_TIMESTAMP,
                 y=oracle_crv.REVERSE_PRICE,
                 color="FEED_NAME",
                title="ETH/CRV Price")
# fig.update_layout(autotypenumbers='convert types')
fig.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_initial_liquidation_end'], 
            annotation_text='Liquidations', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_last_liquidation_start'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)
fig.show()

In [ ]:
fs.dataframes['oracles'].FEED_NAME.unique()

In [13]:
oracle_set = oracles[oracles["FEED_NAME"].isin(['DAI / ETH'])]


In [14]:
oracle_set = oracles[oracles["FEED_ADDRESS"].isin([dai_address, crv_address])]


In [ ]:
oracle_set.head(20)

In [ ]:
crv_val = oracle_set['LATEST_ANSWER_UNADJ'].loc[oracle_set.index[4517]]
dai_val = oracle_set['LATEST_ANSWER_UNADJ'].loc[oracle_set.index[4518]]

adj_crv_val = crv_val /10**18
adj_dai_val = dai_val /10**18

print(adj_crv_val)
print(adj_dai_val)
# CRV / ETH
# ETH / DAI
print((adj_crv_val/ adj_dai_val))

In [ ]:
oracle_set['LATEST_ANSWER_UNADJ'].loc[oracle_set.index[4518]]

In [18]:
def process_oracle_adj(
        target, 
        target_oracle_address, 
        dai_oracle_address = '0x773616E4d11A78F511299002da57A0a94577F1f4'.lower()
    ):
    block_records = {
        'block_number': [], 
        f"{target}_price": [], 
        'dai_price': [], 
        'eth_price': [],
        f"{target}_priced_in_dai": [], 
        'block_timestamp': []
        }
    for index, row in oracle_set.iterrows():
        block_number = row['BLOCK_NUMBER'] 
        block_timestamp = row['BLOCK_TIMESTAMP']
        if row['FEED_ADDRESS'] == target_oracle_address:
            crv = row['LATEST_ANSWER_UNADJ'] / 10**18
            dai = None
        elif row['FEED_ADDRESS'] == dai_oracle_address:
            dai = row['LATEST_ANSWER_UNADJ'] / 10**18
            crv = None
        else: 
            dai = None
            crv = None
        # print (block_records)
        # print(block_number)
        if len(block_records['block_number']) == 0:
            block_records['block_number'].append(block_number)
            block_records['block_timestamp'].append(block_timestamp)
        elif not block_number == block_records['block_number'][-1]:
            block_records['block_number'].append(block_number)
            block_records['block_timestamp'].append(block_timestamp)

        if crv:
            block_records[f"{target}_price"].append(crv)
        elif dai:
            block_records['dai_price'].append(dai)
            block_records['eth_price'].append(1/dai)

        if len(block_records[f"{target}_price"]) == len(block_records['dai_price']) and len(block_records[f"{target}_price"]) > 0:
            last_crv_price = block_records[f"{target}_price"][-1]
            last_dai_price = block_records['dai_price'][-1]
            block_records[f"{target}_priced_in_dai"].append(last_crv_price/last_dai_price)
    return block_records

In [19]:
usd_ref = '0x773616E4d11A78F511299002da57A0a94577F1f4'.lower()

class Oracle():
    def __init__(self, flipside, oracle_feeds_lookup, 
                usd_reference_address=usd_ref
        ):
        self.fs = flipside
        self.oracle_feeds_lookup = oracle_feeds_lookup
        self.usd_reference_address = usd_reference_address
        self.oracles = None
        self.records = None
        self.process_oracles()


    def process_oracles(self):
        oracles = fs.dataframes['oracles']
        records = {}
        for target_address in self.oracle_feeds_lookup:
            target = self.oracle_feeds_lookup[target_address]
            oracle_set = oracles[oracles["FEED_ADDRESS"].isin([self.usd_reference_address, target_address])]
            record = self.process_oracle_helper(oracle_set, target, target_address)
            records[target] = record
        self.records = records
        self.oracles = oracles


    def process_oracle_helper(
            self,
            oracle_set,
            target, 
            target_oracle_address, 
        ):
        block_records = {
            'block_number': [], 
            "eth_price": [], 
            'usd_price_in_eth': [], 
            "usd_price": [], 
            'block_timestamp': []
            }
        for index, row in oracle_set.iterrows():
            block_number = row['BLOCK_NUMBER'] 
            block_timestamp = row['BLOCK_TIMESTAMP']
            if row['FEED_ADDRESS'] == target_oracle_address:
                asset = row['LATEST_ANSWER_UNADJ'] / 10**18
                usd = None
            elif row['FEED_ADDRESS'] == self.usd_reference_address:
                usd = row['LATEST_ANSWER_UNADJ'] / 10**18
                asset = None
            else: 
                usd = None
                asset = None
            # print (block_records)
            # print(block_number)
            if len(block_records['block_number']) == 0:
                block_records['block_number'].append(block_number)
                block_records['block_timestamp'].append(block_timestamp)
            elif not block_number == block_records['block_number'][-1]:
                block_records['block_number'].append(block_number)
                block_records['block_timestamp'].append(block_timestamp)

            if asset:
                block_records['eth_price'].append(asset)
            elif usd:
                block_records['usd_price_in_eth'].append(usd)

            if len(block_records['eth_price']) == len(block_records['usd_price_in_eth']) and len(block_records['eth_price']) > 0:
                last_asset_price = block_records['eth_price'][-1]
                last_usd_price = block_records['usd_price_in_eth'][-1]
                block_records['usd_price'].append(last_asset_price/last_usd_price)
        return block_records

    def get_price(self, target, block_number, min_id=0):
        record = self.records[target]
        section = record['block_number'][min_id:len(record)]
        i = 0
        for b in section:
            if b > block_number:
                break
            i += 1
        target_id = min_id + i
        return {'eth_price': record["eth_price"][target_id],
                'usd_price': record["usd_price"][target_id],
                'symbol': target,
                'last_id': target_id
                }


In [20]:
oracle = Oracle(fs, oracle_feeds_lookup)

In [ ]:
oracle.get_price('1INCH', 16073841)
# oracle.records

In [22]:
# for i, r in fs.dataframes['oracles']:
#     print(i)

In [23]:
block_records = process_oracle_adj('CRV', crv_address, dai_address)

In [ ]:
print(len(block_records['block_number']))
print(len(block_records['block_timestamp']))
print(len(block_records['dai_price']))
print(len(block_records['crv_price']))
print(len(block_records['crv_priced_in_dai']))

In [ ]:
# df = pd.DataFrame(oracle.records['CRV'])
df = pd.DataFrame(block_records)
df

In [ ]:
0.000643/ 0.000888
a = [1,2, 3]
a.append(4)
a[-1]
df.keys()


In [ ]:
fig = px.line(df, 
                 x=df.block_timestamp,
                 y=df.eth_price,
                title="ETH/DAI Price")
# fig.update_layout(autotypenumbers='convert types')
fig.show()

In [ ]:
fig = px.line(df, 
                 x=df.block_timestamp,
                 y=df.CRV_priced_in_dai,
                title="CRV/DAI Price")
# fig.update_layout(autotypenumbers='convert types')
fig.add_vrect(x0=timestamp_windows['avi_attack_start'], x1=timestamp_windows['avi_attack_end'], 
            annotation_text='Attack', annotation_position="top left",
            fillcolor='red', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_initial_liquidation_end'], 
            annotation_text='Liquidations', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_last_liquidation_start'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)
fig.add_vrect(x0='2022-11-22 13:00:00.000', x1='2022-11-22 13:10:00.000', 
            annotation_text='LLAMMA', annotation_position="top left",
            fillcolor='black', opacity=0.25, line_width=0)
fig.show()

In [ ]:
print(0.7239 / 0.4123) # Attack Low to Liquidation High

print(0.609 / 0.6004) # Liq 1

print( 0.7239 / 0.6028 ) # Liq 2


In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])
fig.update_layout(
    title=f"Price Feeds",
#     xaxis_title="X Axis Title",
#     yaxis_title="Y Axis Title",
#     legend_title="Legend Title",
    font=dict(
        family="Courier New, monospace",
        size=18,
        color="RebeccaPurple"
    )
)
# fig = fig.add_trace(
#     go.Scatter(
#         x=df.block_timestamp,
#         y=df.crv_price,
#         name="CRV/ETH Price",
#         mode='lines',
#         marker_color="green",
#     ),
#     secondary_y=False
# )

fig = fig.add_trace(
    go.Scatter(
        x=df.block_timestamp,
        y=df.CRV_priced_in_dai,
        name="CRV/DAI Price",
        mode='lines',
        marker_color="blue",
    ),
    secondary_y=False
)

fig = fig.add_trace(
    go.Scatter(
        x=df.block_timestamp,
        y=df.eth_price,
        name="ETH/DAI Price",
        mode='lines',
        marker_color="red",
    ),
    secondary_y=True
)


fig.add_vrect(x0=timestamp_windows['avi_initial_deposit'], x1=timestamp_windows['avi_attack_start'], 
            annotation_text='A Setup', annotation_position="top left",
            fillcolor='orange', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_attack_start'], x1=timestamp_windows['avi_initial_liquidation'], 
            annotation_text='Attack', annotation_position="top left",
            fillcolor='blue', opacity=0.25, line_width=0)

fig.add_vrect(x0=timestamp_windows['avi_initial_liquidation'], x1=timestamp_windows['avi_last_liquidation'], 
            annotation_text='', annotation_position="top left",
            fillcolor='purple', opacity=0.25, line_width=0)



fig.add_vrect(x0=timestamp_windows['michael_ramp_up'], x1=timestamp_windows['michael_ramp_stop'], 
            annotation_text='M Setup', annotation_position="top left",
            fillcolor='blue', opacity=0.25, line_width=0)

# fig.update_xaxes(range=[1.5, 4.5])
fig.update_yaxes(range=[0.30, 1.10])
fig.update_yaxes(range=[463, 1700], secondary_y=True)

fig.show()

In [ ]:
(0.3 / 1.1) * 1700
